In [0]:
print("Healthy")

In [0]:
from re import search
from sqlalchemy import create_engine

import pandas as pd 
import numpy as np 

import mysql.connector

import warnings
warnings.filterwarnings('ignore')


In [0]:
from datetime import datetime, timedelta
today = datetime.today().strftime('%Y-%m-%d')
yesterday =  (datetime.today() - timedelta(days = 1)).strftime('%Y-%m-%d')
time_now = datetime.today().strftime('%Y-%m-%d %H')
prev_time = (datetime.today() - timedelta(days = 1)).strftime('%Y-%m-%d 18')

print(prev_time, time_now)
print(yesterday, today)
print(len(prev_time))

In [0]:
import itables

from re import search
from sqlalchemy import create_engine
import duckdb

import pandas as pd 
import numpy as np 
import mysql.connector

#from custom_packages.email_operations import EmailOperations


import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)


import json
with open('/Workspace/Credentials/db_data.json', 'r') as fp:
    data = json.load(fp)

Dumpdb = mysql.connector.connect(
  host=data['Dumpdb']['host'],
  user=data['Dumpdb']['user'],
  passwd=data['Dumpdb']['passwd'],
  database = data['Dumpdb']['database']
)


Mifosdb = mysql.connector.connect(
  host=data['mifos']['host'],
  user=data['mifos']['user'],
  passwd=data['mifos']['passwd'],
  database = data['mifos']['database']
)


vbizdb = mysql.connector.connect(
     host=data['vbizdb']['host'],
  user=data['vbizdb']['user'],
  passwd=data['vbizdb']['passwd'],
  database = data['vbizdb']['database']
)


host = data['redshift']['host']
user = data['redshift']['user']
passwd = data['redshift']['passwd']
database = data['redshift']['database']

conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")
pd.set_option('display.float_format', lambda x: '%.2f' % x)



In [0]:
username = data['auto-report']['email']
passwd = data['auto-report']['passwd']
print(username)

In [0]:
bal = '''
SELECT
mc.display_name AS client_name,
sa.account_no,
sp.name AS product_name,
sa.account_balance_derived,
MAX(st.created_date) AS last_transaction_date,
st.running_balance_derived AS balance_as_at_last_transaction
FROM m_savings_account sa
JOIN m_client mc ON mc.id = sa.client_id
JOIN m_savings_product sp ON sp.id = sa.product_id
LEFT JOIN m_savings_account_transaction st ON st.savings_account_id = sa.id
WHERE sp.id = 136
GROUP BY sa.account_no
'''

dfbal = pd.read_sql_query(bal, Mifosdb)

dfbal

In [0]:
with pd.ExcelWriter(f"/Workspace/ReportDump/Cashex_Subaccount/Cashex subaccount daily balance as at {today}.xlsx") as writer:
    dfbal.to_excel(writer, sheet_name = f'cashex subaccount balances', index = False)
    

In [0]:
# FILE TO SEND AND ITS PATH
filename = f'Cashex subaccount daily balance as at {today}.xlsx'
SourcePathName  = "/Workspace/ReportDump/Cashex_Subaccount/" + filename


In [0]:
import csv, smtplib, ssl
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders



msg = MIMEMultipart()
msg['From'] = 'auto-report@vfdtech.ng'
# msg['To'] = 'Henry.Nnonyelu@vfdtech.ng'
msg['To'] = "treasury@cashex.app, scott@cashex.app"
msg['cc'] = 'Henry.Nnonyelu@vfdtech.ng, Olugbenga.Paseda@vfdtech.ng'
msg['Bcc'] = 'data-team@vfdtech.ng'
msg['Subject'] = f'CASHEX SUBACCOUNT BALANCES'
body = f"""
Hello Cashex,

Please find attached report for subaccount balances as at {today}.

Regards,
"""
msg.attach(MIMEText(body, 'plain'))



## ATTACHMENT PART OF THE CODE IS HERE
attachment = open(SourcePathName, 'rb')
part = MIMEBase('application', "octet-stream")
part.set_payload((attachment).read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', "attachment; filename= %s" % filename)
msg.attach(part)

server = smtplib.SMTP('smtp.office365.com', 587)  ### put your relevant SMTP here
server.ehlo()
server.starttls()
server.ehlo()
server.login(username, passwd)  ### if applicable
server.send_message(msg)
server.quit()